# A Data-Driven AMPL Model

In this notebook, we'll revisit the production planning example. However, this time we'll demonstrate how Python's data structures combine with AMPL's ability to separate model and data, to create an optimization model that scales with the size of the data tables. This enables the model to adjust to new products, varying prices, or changing demand. We refer to this as "data-driven" modeling.

This notebook introduces two new AMPL model components that describe the data in a general way:

* Sets
* Parameters

These components enable the model to specify variables, constraints, and summations that are *indexed* over sets. The combination of sets and indices is essential to building scalable and maintainable models for more complex applications.

We will begin this analysis by examining the production planning data sets to identify the underlying problem structure. Then we will reformulate the mathematical model in a more general way that is valid for any data scenario. Finally we show how the same formulation carries over naturally into AMPL, providing a clear, data-driven formulation of the production planning application.

## Data representations

We begin by revisiting the data tables and mathematical model developed for the basic production planning problem presented in the previous notebook. The original data values were given as follows:

| Product | Material <br> required | Labor A <br> required | Labor B <br> required | Market <br> Demand | Price |
| :-: | :-: | :-: | :-: | :-: | :-: |
| U | 10 g | 1 hr | 2 hr | $\leq$ 40 units | \$270 |
| V |  9 g | 1 hr | 1 hr | unlimited | \$210 |

| Resource | Amount <br> Available | Cost |
| :-: | :-: | :-: |
| M | unlimited | \$10 / g |
| A | 80 hours | \$50 / hour |
| B | 100 hours | \$40 / hour |

Two distinct *sets* of objects are evident from these tables. The first is the set of products, comprising $U$ and $V$. The second is the set of resources used to produce those products, comprising raw materials and two labor types, which we have abbreviated as $M$, $A$, and $B$.

Having identified these sets, the data for this application be factored into three simple tables. The first two tables list attributes of the products and attributes of the resources. The third table summarizes the processes used to create the products from the resources, which requires providing a value for each combination of product and resource:

**Table: Products**

| Product | Demand | Price |
| :-: | :-: | :-: |
| U |  $\leq$ 40 units | \$270 |
| V |  unlimited | \$210 |

**Table: Resources**

| Resource | Available | Cost |
| :-: | :-: | :-: |
| M | ? | \$10 / g |
| A | 80 hours | \$50 / hour |
| B | 100 hours | \$40 / hour |

**Table: Processes**

| Product | M | A | B
| :-: | :-: | :-: | :-: |
| U | 10 g | 1 hr | 2 hr |
| V |  9 g | 1 hr | 1 hr |


How does a Python-based AMPL application work with this data? We can think of the data as being handled in three steps:

1. Import the data into Python, in whatever form is convenient for the application.

1. Convert the data to the forms required by the optimization model.

1. Send the data to AMPL.

For our example, we implement step 1 by use of Python *nested dictionaries* that closely resemble the above three tables:

- In the `products` data structure, the product abbreviations serve as keys for outermost dictionary, and the product-related attribute names (`demand` and `price`) as keys for the inner dictionaries.

- In the `resources` data structure, the resource abbreviations serve as keys for outermost dictionary, and the resource-related attribute names (`available` and `cost`) as keys for the inner dictionaries.

- In the `processes` data structure, there is a value corresponding to each combination of a product and a resource; the product abbreviations serve as keys for outermost dictionary, and resource abbreviations as keys for the inner dictionaries.

Where demand or availability is "unlimited", we use an expression that Python interprets as an infinite value.

You will see a variety of data representation in this book, chosen in each case to be most efficient and convenient for the application at hand. Some will use Python packages, particularly *numpy* and *pandas*, that are designed for large-scale data handling.

In [2]:
Inf = float("inf")

products = {
    "U": {"demand": 40, "price": 270},
    "V": {"demand": Inf, "price": 210},
}

resources = {
    "M": {"available": Inf, "cost": 10},
    "A": {"available": 80, "cost": 50},
    "B": {"available": 100, "cost": 40},
}

processes = {
    "U": {"M": 10, "A": 2, "B": 1},
    "V": {"M": 9, "A": 1, "B": 1},
}

## Modelo matemático

Una vez que los datos del problema se reorganizan en tablas como estas, la estructura del problema de planificación de la producción se vuelve evidente. Junto con los dos conjuntos, tenemos una variedad de *parámetros* simbólicos que especifican los costos, límites y procesos del modelo de manera general. En comparación con el cuaderno anterior, estas abstracciones nos permiten crear modelos matemáticos que pueden adaptarse y escalar con los datos suministrados.

Sean $\cal{P}$ y $\cal{R}$ el conjunto de productos y recursos, respectivamente, y sean $p$ y $r$ elementos representativos de esos conjuntos. Usamos variables de decisión indexadas $x_r$ para denotar la cantidad de recurso $r$ que se consume en la producción, y $y_p$ para denotar la cantidad de producto $p$ producido.

El modelo especifica límites inferiores y superiores para los valores de las variables. Representamos estos límites como:

$$
\begin{aligned}
    0 \leq x_r \leq b^x_r & & \forall r\in\cal{R} \\
    0 \leq y_p \leq b^y_p & & \forall p\in\cal{P} \\
\end{aligned}
$$

donde los límites superiores, \(b^x_r\) y \(b^y_p\), son datos tomados de las tablas de atributos.

La función objetivo es dada por: 

$$
\begin{aligned}
    \text{profit} & = \text{revenue} - \text{cost} \\
\end{aligned}
$$

pero ahora las expresiones para ingresos y costos se expresan de manera más general como sumas sobre los conjuntos de productos y recursos,

$$
\begin{aligned}
    \text{revenue} & = \sum_{p\in\cal{P}} c^y_p y_p  \\
    \text{cost} & = \sum_{r\in\cal{R}} c^x_r x_r \\
\end{aligned}
$$

donde los parámetros \(c^y_p\) y \(c^x_r\) representan los precios de venta de los productos y los costos de los recursos, respectivamente. Los límites en los recursos disponibles se pueden escribir como

$$
\begin{aligned}
    \sum_{p\in\cal{P}} a_{rp} y_p & \leq x_r & \forall r\in\cal{R}
\end{aligned}
$$

donde \(a_{rp}\) es la cantidad de recurso \(r\) necesaria para producir 1 unidad del producto \(p\). Juntando estas piezas, tenemos el siguiente modelo simbólico para el problema de planificación de la producción.

$$
\begin{align}
{\rm maximize} \quad & \sum_{p\in\cal{P}} c^y_p y_p - \sum_{r\in\cal{R}} c^x_r x_r \\
\text{subject to} \quad & \sum_{p\in\cal{P}} a_{rp} y_p  \leq x_r & \forall r\in\cal{R} \nonumber \\
 &   0 \leq x_r \leq b^x_r & \forall r\in\cal{R} \nonumber  \\
 &   0 \leq y_p \leq b^y_p & \forall p\in\cal{P} \nonumber  \\
\end{align}
$$

Cuando se formula de esta manera, el modelo puede aplicarse a cualquier problema con la misma estructura, independientemente del número de productos o recursos. Esta flexibilidad es posible gracias al uso de conjuntos para describir los productos y recursos de una instancia particular del problema, índices como \(p\) y \(r\) para referirse a elementos de esos conjuntos, y tablas de datos que contienen los valores de los parámetros relevantes.

Generalizar los modelos matemáticos de esta manera es una característica de todas las aplicaciones de optimización a gran escala. A continuación, veremos cómo este tipo de generalización se traslada de forma natural a la formulación y resolución del modelo en AMPL.

## El modelo de producción en AMPL
Como antes, comenzamos la construcción de un modelo en AMPL importando los componentes necesarios en el entorno de AMPL.

In [3]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="cf36ebc4-be04-47f5-9536-c60cc47c7341",  # license to use
)  # instantiate AMPL object and register magics

Licensed to Bundle #6719.7170 expiring 20241231: Network Flow and Transportation Analytics, Prof. Alejandra Tabares, Universidad de los Andes - Colombia.


A continuación, utilizamos las declaraciones `set` de AMPL para definir los conjuntos de productos y recursos. Es importante notar que, en este punto, solo estamos informando a AMPL sobre los dos conjuntos que se utilizarán en el modelo. Los miembros de estos conjuntos se enviarán desde Python a AMPL más adelante, como parte de los datos del problema.

En las formulaciones matemáticas, es habitual mantener los nombres de todos los componentes cortos. Pero al escribir el modelo en AMPL, somos libres de usar nombres más largos y significativos que hagan que las declaraciones del modelo sean más fáciles de leer. Así, por ejemplo, aquí usamos `PRODUCTS` y `RESOURCES` como los nombres de los conjuntos en AMPL que se llaman $\cal P$ y $\cal R$ en el modelo matemático.

In [4]:
%%ampl_eval
# define sets

set PRODUCTS;
set RESOURCES;

El siguiente paso es introducir los parámetros que se usarán como datos en la función objetivo y las restricciones.

Una declaración que define un parámetro en AMPL comienza con la palabra clave `param` y un nombre único. Luego, entre llaves `{` y `}` especifica los conjuntos de índices para el parámetro. Por ejemplo:

- `param demand {PRODUCTS} >= 0;` indica que hay un valor de "demanda de producto" para cada miembro del conjunto `PRODUCTS`.

- `param need {RESOURCES,PRODUCTS} >= 0;` indica que hay un valor de "recurso necesario" para cada combinación de un recurso y un producto.

Al final de cada declaración `param`, especificamos que los valores del parámetro deben ser no negativos o positivos, según corresponda. Estas especificaciones se usarán más adelante para verificar que los valores de los datos reales sean apropiados para el problema.

Hay 5 declaraciones `param` diferentes en total, correspondientes a los 5 tipos diferentes de datos en las tablas y los 5 parámetros simbólicos $b_p^y$, $c_p^y$, $b_r^x$, $c_r^x$, y $a_{rp}$ en el modelo matemático.

In [5]:
%%ampl_eval
# define parameters

param demand {PRODUCTS} >= 0;
param price {PRODUCTS} > 0;

param available {RESOURCES} >= 0;
param cost {RESOURCES} > 0;

param need {RESOURCES,PRODUCTS} >= 0;

AMPL define las variables de decisión de manera muy similar a los parámetros, pero utilizando `var` como palabra clave para iniciar la declaración. Nombramos las variables `Use` para el uso de recursos y `Sell` para las ventas de productos.

Para expresar los límites de las variables de la misma manera que la formulación matemática, se necesita una forma más general de la declaración de AMPL. En el caso de las variables `Use`, por ejemplo:

* La expresión de indexación se escribe `{r in RESOURCES}` para indicar que hay una variable para cada miembro del conjunto de recursos, y también para asociar el *índice* `r` con los miembros del conjunto para el propósito de esta declaración. Esto es el equivalente en AMPL de $\forall r\in\cal{R}$ en la declaración matemática.

* El límite superior se escribe `<= available[r]` para indicar que para cada miembro `r` del conjunto de recursos, el límite superior de la variable está dado por el valor correspondiente de la tabla de disponibilidad. Esto es el equivalente en AMPL de $\leq b^x_r$ en la declaración matemática.

Una expresión entre corchetes `[...]` se llama un *subíndice* en AMPL porque juega el mismo papel que un subíndice matemático como $r$ en $\leq b^x_r$. En cualquier lugar donde el modelo se refiere a valores particulares de un parámetro o variable indexados, verás expresiones de subíndice. Por ejemplo,

* `need[r,p]` será la cantidad de recurso `r` necesaria para producir una unidad del producto `p`.

* `Use[r]` será la cantidad total de recurso `r` utilizada.

In [6]:
%%ampl_eval
# define variables

var Use {r in RESOURCES} >= 0, <= available[r];
var Sell {p in PRODUCTS} >= 0, <= demand[p];

Al igual que en el cuaderno anterior, la declaración de AMPL para la función objetivo comienza con `maximize Profit`. Pero ahora, como en la formulación matemática, AMPL utiliza expresiones de sumación general:

* `sum {p in PRODUCTS} price[p] * Sell[p]` es la suma, sobre todos los productos, del precio por unidad multiplicado por el número vendido. Esto corresponde a $\sum_{p\in\cal{P}} c^y_p y_p$ en la formulación matemática.

* `sum {r in RESOURCES} cost[r] * Use[r]` es la suma, sobre todos los recursos, del costo por unidad multiplicado por la cantidad utilizada. Esto corresponde a $\sum_{r\in\cal{R}} c^x_r x_r$ en la formulación matemática.

La expresión completa para la función objetivo es simplemente la primera de estas expresiones menos la segunda.

In [7]:
%%ampl_eval
# define objective function

maximize Profit:
   sum {p in PRODUCTS} price[p] * Sell[p] -
   sum {r in RESOURCES} cost[r] * Use[r];

El modelo de AMPL anterior tenía 3 restricciones, cada una definida por una declaración `subject to`. Pero la formulación matemática basada en datos reconoce que solo hay un tipo diferente de restricción: los recursos necesarios deben ser menores o iguales a los recursos utilizados, repetida 3 veces, una vez para cada recurso. La versión de AMPL combina expresiones que ya han aparecido en partes anteriores del modelo:

* `subject to ResourceLimit {r in RESOURCES}` indica que el modelo tendrá una restricción correspondiente a cada miembro `r` del conjunto de recursos.

* `sum {p in PRODUCTS} need[r,p] * Sell[p] <= Use[r]` indica que el total del recurso `r` necesario, sumado sobre todos los productos vendidos, debe ser `<=` el total del recurso `r` utilizado. Esto corresponde a $\sum_{p\in\cal{P}} a_{rp} y_p \leq x_r$ en la formulación matemática.

In [8]:
%%ampl_eval
# create indexed constraint

subject to ResourceLimit {r in RESOURCES}:
   sum {p in PRODUCTS} need[r,p] * Sell[p] <= Use[r];

## Los datos de producción en AMPL

Ahora que el modelo de AMPL está definido, podemos llevar a cabo el paso 2 de manejo de datos, que es convertir los datos a las formas que el modelo requiere:

- Para los dos conjuntos, listas de Python de los miembros del conjunto.
- Para los dos parámetros indexados sobre productos, diccionarios de Python cuyas claves son los nombres de los productos.
- Para los dos parámetros indexados sobre recursos, diccionarios de Python cuyas claves son los nombres de los recursos.
- Para el parámetro indexado sobre pares recurso-producto, un diccionario de Python cuyas claves son tuplas que consisten en un recurso y un producto.

Utilizando las poderosas formas de expresión de Python, todas estas listas y diccionarios se extraen fácilmente de los diccionarios anidados que nuestra aplicación configuró en el paso 1. Para evitar tener demasiados nombres diferentes, asignamos cada lista y diccionario a una variable de programa de Python que tiene el mismo nombre que el conjunto o parámetro correspondiente en AMPL:

In [9]:
# set data
PRODUCTS = products.keys()
RESOURCES = resources.keys()

# product data
demand = {k: v["demand"] for k, v in products.items()}
price = {k: v["price"] for k, v in products.items()}

# resource data
available = {k: v["available"] for k, v in resources.items()}
cost = {k: v["cost"] for k, v in resources.items()}

need = {(r, p): value for p in processes.keys() for r, value in processes[p].items()}

print(PRODUCTS, RESOURCES)
print(demand, price)
print(available, cost)
print(need)

dict_keys(['U', 'V']) dict_keys(['M', 'A', 'B'])
{'U': 40, 'V': inf} {'U': 270, 'V': 210}
{'M': inf, 'A': 80, 'B': 100} {'M': 10, 'A': 50, 'B': 40}
{('M', 'U'): 10, ('A', 'U'): 2, ('B', 'U'): 1, ('M', 'V'): 9, ('A', 'V'): 1, ('B', 'V'): 1}


## Resolviendo el problema de producción

Ahora los datos de Python pueden ser enviados a AMPL, y AMPL puede invocar un solver. Para este modelo simple, podemos hacer que los datos de Python correspondan exactamente a los datos de AMPL, por lo que las sentencias para enviar los datos a AMPL son particularmente fáciles de escribir.

Las sentencias para seleccionar un solver e iniciar el proceso del solver son las mismas que usamos con el ejemplo básico de planificación de la producción. Cuando el solver termina, muestra algunas líneas de salida para confirmar que se ha encontrado una solución.

In [10]:
# load set data
ampl.set["PRODUCTS"] = PRODUCTS
ampl.set["RESOURCES"] = RESOURCES

# load parameter data
ampl.param["price"] = price
ampl.param["demand"] = demand
ampl.param["cost"] = cost
ampl.param["available"] = available
ampl.param["need"] = need

# set solver and solve
ampl.option["solver"] = "highs"
ampl.solve()

HiGHS 1.7.0:HiGHS 1.7.0: optimal solution; objective 2400
2 simplex iterations
0 barrier iterations


## Reportando los resultados

Resta recuperar la solución de AMPL, después de lo cual se pueden utilizar las amplias características y el ecosistema de Python para presentar los resultados de la manera deseada. Para este primer ejemplo, utilizamos una de las características más simples de Python, la declaración `print`.

Una entidad de AMPL se referencia en el código Python a través de su nombre en el modelo de AMPL. Por ejemplo, la función objetivo `Profit` es `ampl.obj['Profit']`, y la colección de variables `Sell` es `ampl.var['Sell']`.

Para una entidad que no está indexada, el método `value()` devuelve el valor asociado. Así, la primera declaración `print` se refiere a `ampl.obj['Profit'].value()`.

Para una entidad indexada, usamos el método `to_dict()` para devolver los valores en un diccionario de Python, con los miembros del conjunto como claves. Luego, un bucle `for` puede usar el método `items()` para iterar sobre el diccionario y imprimir una línea para cada miembro.

In [11]:
# create a solution report
print(f"Profit = {ampl.obj['Profit'].value()}")

print("\nProduction Report")
for product, Sell in ampl.var["Sell"].to_dict().items():
    print(f" {product} produced = {Sell}")

print("\nResource Report")
for resource, Use in ampl.var["Use"].to_dict().items():
    print(f" {resource} consumed = {Use}")

Profit = 2400.0

Production Report
 U produced = 0
 V produced = 80

Resource Report
 A consumed = 80
 B consumed = 80
 M consumed = 720


## Para expertos en Python: Creación de subclases de `AMPL`

Algunos lectores de estos cuadernos pueden ser desarrolladores de Python más experimentados que deseen aplicar AMPL en aplicaciones más especializadas y basadas en datos. La siguiente celda muestra cómo la clase AMPL puede ser extendida para crear clases de modelos especializadas. Aquí creamos una subclase llamada `ProductionModel` que acepta una representación particular de los datos del problema para producir un objeto de modelo de producción. El objeto del modelo de producción hereda todos los métodos asociados con cualquier modelo de AMPL, como `.display()` y `.solve()`, pero puede ser extendido con métodos adicionales.

In [12]:
%%writefile production_planning.mod

# define sets
set PRODUCTS;
set RESOURCES;

# define parameters
param demand {PRODUCTS} >= 0;
param price {PRODUCTS} > 0;
param available {RESOURCES} >= 0;
param cost {RESOURCES} > 0;
param need {RESOURCES,PRODUCTS} >= 0;

# define variables
var Use {r in RESOURCES} >= 0, <= available[r];
var Sell {p in PRODUCTS} >= 0, <= demand[p];

# define objective function
maximize Profit:
   sum {p in PRODUCTS} price[p] * Sell[p] -
   sum {r in RESOURCES} cost[r] * Use[r];

# create indexed constraint
subject to ResourceLimit {r in RESOURCES}:
   sum {p in PRODUCTS} need[r,p] * Sell[p] <= Use[r];

Writing production_planning.mod


In [13]:
import pandas as pd


class ProductionModel(AMPL):
    """
    A class representing a production model using AMPL.
    """

    def __init__(self, products, resources, processes):
        """
        Initialize ProductionModel as an AMPL instance.

        :param products: A dictionary containing product information.
        :param resources: A dictionary containing resource information.
        :param processes: A dictionary containing process information.
        """
        super(ProductionModel, self).__init__()

        # save data in the model instance
        self.products = products
        self.resources = resources
        self.processes = processes

        # flag to monitor solution status
        self.solved = False

    def load_data(self):
        """
        Prepare the data and pass the information to AMPL.
        """
        # convert the data dictionaries into pandas data frames
        products = pd.DataFrame(self.products).T
        resources = pd.DataFrame(self.resources).T
        processes = pd.DataFrame(self.processes).T

        # display the generated data frames
        display(products)
        display(resources)
        display(processes)

        # pass data to AMPL
        self.set_data(products, "PRODUCTS")
        self.set_data(resources, "RESOURCES")
        self.param["need"] = processes.T

    def solve(self, solver="highs"):
        """
        Read the model, load the data, set the solver and solve the optimization problem.
        """
        self.read("production_planning.mod")
        self.load_data()
        self.option["solver"] = solver
        super(ProductionModel, self).solve()
        self.solved = True

    def report(self):
        """
        Solve, if necessary, then report the model solution.
        """
        if not self.solved:
            self.solve()

        print(f"Profit = {self.obj['Profit'].value()}")

        print("\nProduction Report")
        Sell = self.var["Sell"].to_pandas()
        Sell.rename(columns={Sell.columns[0]: "produced"}, inplace=True)
        Sell.index.rename("PRODUCTS", inplace=True)
        display(Sell)

        print("\nResource Report")
        Use = self.var["Use"].to_pandas()
        Use.rename(columns={Use.columns[0]: "consumed"}, inplace=True)
        Use.index.rename("RESOURCES", inplace=True)
        display(Use)


m = ProductionModel(products, resources, processes)
m.report()

,demand,price
U,40.0,270.0
V,inf,210.0


,available,cost
M,inf,10.0
A,80.0,50.0
B,100.0,40.0


,M,A,B
U,10,2,1
V,9,1,1


HiGHS 1.7.0:HiGHS 1.7.0: optimal solution; objective 2400
2 simplex iterations
0 barrier iterations
Profit = 2400.0

Production Report


,produced
PRODUCTS,
U,0
V,80



Resource Report


,consumed
RESOURCES,
A,80
B,80
M,720
